Input a Postal Code, Output Polygon Plot & Polygon Data

In [1]:
import json
from shapely.geometry import shape, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
from pymongo import MongoClient

def connect_to_mongodb(database_name, collection_name):
    """
    Connects to MongoDB and returns the MongoDB client and collection.
    """
    client = MongoClient('localhost', 27017)
    db = client[database_name]
    collection = db[collection_name]
    return client, collection

def retrieve_data_from_mongodb(collection):
    """
    Retrieves GeoJSON data from MongoDB collection.
    """
    cursor = collection.find()
    geojson_data = {"type": "FeatureCollection", "features": []}

    for document in cursor:
        geojson_data["features"].append(document)

    return geojson_data

def coordinates_to_polygon(coordinates):
    """
    Converts coordinates to a Shapely Polygon.
    """
    return Polygon(coordinates)

def plot_polygon(postal_code_polygons, postal_code):
    """
    Plots the polygon for a given postal code and returns the polygon.
    """
    polygon = postal_code_polygons.get(postal_code)
    if polygon:
        gdf = gpd.GeoDataFrame(geometry=[polygon])
        gdf.plot()
        plt.title(f"Postal Code Polygon: {postal_code}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()
        return polygon
    else:
        print(f"No polygon found for postal code: {postal_code}")
        return None

# Example usage
if __name__ == "__main__":
    # MongoDB configuration
    database_name = "ZIP_Poly_Ger_Test"
    collection_name = "ZIP_Poly_Ger_Collection_Test"

    # Connect to MongoDB
    mongo_client, mongo_collection = connect_to_mongodb(database_name, collection_name)

    # Retrieve GeoJSON data from MongoDB
    geojson_data_from_mongo = retrieve_data_from_mongodb(mongo_collection)

    # Load GeoJSON data from MongoDB for comparison
    postal_code_polygons_mongo = {document.get('postal_code'): coordinates_to_polygon(document.get('geometry')['coordinates'][0]) for document in geojson_data_from_mongo['features']}

    # Input the postal code you want to plot
    input_postal_code = input("Enter a postal code: ")

    # Plot the polygon for the specified postal code and get the polygon
    selected_polygon = plot_polygon(postal_code_polygons_mongo, input_postal_code)

    # Output the polygon information
    if selected_polygon:
        print("Selected Polygon Information:")
        print(selected_polygon)

    # Close MongoDB connection when done
    mongo_client.close()


TypeError: 'float' object is not iterable